In [2]:
import datetime as dt
import os
import pandas as pd
import pickle
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np


In [3]:
style.use('ggplot')

### Unione dati

In [4]:
with open('crypto_ticker.pickle', 'rb') as f:
    tickers = pickle.load(f)

In [5]:
df = pd.read_csv('crypto/{}.csv'.format('BTC'))
df.set_index('Date', inplace=True)

df.rename(columns = {'Adj Close': 'BTC'}, inplace=True)
df.drop(['Open','High','Low','Close','Volume'], 1, inplace=True)
df.head()

,BTC
Date,
2014-09-16,457.334015
2014-09-17,424.440002
2014-09-18,394.795990
2014-09-19,408.903992
2014-09-20,398.821014


In [6]:
main_df = pd.DataFrame()
for count, ticker in enumerate(tickers[:30]):
    if count % 10 == 0:
        print(count)

    try:
        df = pd.read_csv('crypto/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns = {'Adj Close': ticker}, inplace=True)
        df.drop(['Open','High','Low','Close','Volume'], 1, inplace=True)

        if main_df.empty:
            main_df = df
        else: 
            main_df = main_df.join(df, how='outer')
    except FileNotFoundError as e:
        pass

0
10
20


In [7]:
main_df.head()

,BTC,ETH,XRP,USDT,BCH,LTC,EOS,BNB,XLM,LINK,TRX,ADA,XMR,DASH,ETC,NEO,MIOTA,XEM,ZEC
Date,,,,,,,,,,,,,,,,,,,
2014-09-16,457.334015,NaN,0.005399,NaN,NaN,5.05855,NaN,NaN,0.003072,NaN,NaN,NaN,1.76835,3.75394,NaN,NaN,NaN,NaN,NaN
2014-09-17,424.440002,NaN,0.005135,NaN,NaN,4.68523,NaN,NaN,0.003456,NaN,NaN,NaN,1.56928,3.86890,NaN,NaN,NaN,NaN,NaN
2014-09-18,394.795990,NaN,0.004942,NaN,NaN,4.32777,NaN,NaN,0.003872,NaN,NaN,NaN,1.45437,3.43210,NaN,NaN,NaN,NaN,NaN
2014-09-19,408.903992,NaN,0.004772,NaN,NaN,4.28644,NaN,NaN,0.003612,NaN,NaN,NaN,1.27121,3.18344,NaN,NaN,NaN,NaN,NaN
2014-09-20,398.821014,NaN,0.004570,NaN,NaN,4.24592,NaN,NaN,0.003464,NaN,NaN,NaN,1.27415,2.97051,NaN,NaN,NaN,NaN,NaN


In [8]:
main_df.tail()

,BTC,ETH,XRP,USDT,BCH,LTC,EOS,BNB,XLM,LINK,TRX,ADA,XMR,DASH,ETC,NEO,MIOTA,XEM,ZEC
Date,,,,,,,,,,,,,,,,,,,
2020-03-15,5392.314941,125.214302,0.153902,1.001987,178.311279,36.437092,2.050833,10.436018,0.038282,2.109211,0.010344,0.027120,38.070606,47.499184,5.056345,6.004697,0.132011,0.034255,25.783978
2020-03-16,5014.479980,110.605873,0.141063,0.997004,170.736649,32.876217,1.884314,9.386050,0.035161,1.779877,0.009561,0.024121,33.816479,42.945484,4.448968,5.377224,0.110195,0.032512,24.504303
2020-03-17,5225.629395,113.942749,0.144524,0.976145,180.326385,33.563869,1.930519,10.077175,0.036820,1.842805,0.009885,0.025243,35.366760,44.619865,4.526495,5.534018,0.120833,0.033534,24.647459
2020-03-18,5238.438477,114.842270,0.143100,0.974248,179.092331,33.684177,1.945424,10.119137,0.035979,1.887700,0.009934,0.025336,35.922001,59.593079,4.472723,5.546127,0.123403,0.034293,28.453094
2020-03-19,6191.192871,136.593857,0.165037,1.003193,221.268555,39.073765,2.290769,12.279640,0.041494,2.246666,0.011668,0.030451,41.103573,65.849304,5.058727,6.509382,0.169805,0.040339,32.052864


In [9]:
main_df.to_csv('crypto_joined_closes.csv')

### Correlazione di Pearson

In [10]:
df = pd.read_csv('crypto_joined_closes.csv')

In [11]:
num_of_crypto = 10

In [12]:
df.set_index('Date', inplace=True)
df_corr = df.pct_change().corr(method='pearson').iloc[:num_of_crypto,:num_of_crypto]

In [13]:
df_corr.head(10)

,BTC,ETH,XRP,USDT,BCH,LTC,EOS,BNB,XLM,LINK
BTC,1.000000,0.463102,0.304028,-0.028070,0.513914,0.593931,0.464074,0.492683,0.323512,0.216099
ETH,0.463102,1.000000,0.254990,-0.003394,0.607011,0.423362,0.540015,0.453720,0.267134,0.258400
XRP,0.304028,0.254990,1.000000,-0.018630,0.427855,0.326780,0.410996,0.291434,0.486692,0.190663
USDT,-0.028070,-0.003394,-0.018630,1.000000,-0.033233,-0.014581,-0.098678,-0.042703,-0.013063,0.001959
BCH,0.513914,0.607011,0.427855,-0.033233,1.000000,0.521935,0.476932,0.321092,0.327758,0.173911
LTC,0.593931,0.423362,0.326780,-0.014581,0.521935,1.000000,0.470965,0.411934,0.308864,0.190671
EOS,0.464074,0.540015,0.410996,-0.098678,0.476932,0.470965,1.000000,0.339560,0.353895,0.198816
BNB,0.492683,0.453720,0.291434,-0.042703,0.321092,0.411934,0.339560,1.000000,0.273869,0.177298
XLM,0.323512,0.267134,0.486692,-0.013063,0.327758,0.308864,0.353895,0.273869,1.000000,0.188228
LINK,0.216099,0.258400,0.190663,0.001959,0.173911,0.190671,0.198816,0.177298,0.188228,1.000000


In [18]:
#%matplotlib auto

data = df_corr.values
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

heatmap = ax.pcolor(data, cmap=plt.cm.RdYlGn) # Red Yellow Green
fig.colorbar(heatmap)

ax.set_xticks(np.arange(data.shape[0]) + 0.5, minor=False)
ax.set_yticks(np.arange(data.shape[1]) + 0.5, minor=False)

ax.invert_yaxis() # così row1 col1 sono uguali, row2, col2 idem, etc.
ax.xaxis.tick_top() # ticks on top e non bottom


column_labels = df_corr.columns
row_labels = df_corr.index

ax.set_xticklabels(column_labels)
ax.set_yticklabels(row_labels)

plt.xticks(rotation=90) # ruoto di 90° i tick sull'asse delle x
heatmap.set_clim(-1,1)

for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        plt.text(x + 0.5, y + 0.5, '%.2f' % data[y, x],
                 horizontalalignment='center',
                 verticalalignment='center')



plt.tight_layout()
plt.show()

In [15]:
%matplotlib auto
import seaborn as sns

#sns.heatmap(df_corr,annot=True,cmap=plt.cm.RdYlGn)
#sns.heatmap(df_corr,annot=True,fmt='.2g',vmin=-1, vmax=1, center= 0, cbar_kws= {'orientation': 'horizontal'})
sns.heatmap(df_corr,annot=True,cmap=plt.cm.RdYlGn,fmt='.2g',vmin=-1, vmax=1, center= 0)
plt.show()

Using matplotlib backend: MacOSX
